# Assignment 6: Natural Language Generation Generation

In this part of the homework, you will implement decoding algorithms covered in class -- greedy decoding, random sampling, temperature sampling, top-k sampling, and top-p (nucleus) sampling. You will also learn how to use knowledge distillation to use strong teacher models to improve performance of weaker student models. The knowledge distillation exercise is only mandatory for CSE 517 students.

Notes about the autograder for this assignment:

- To submit the coding part of this assignment to Gradescope, download your .ipynb notebook in .ipynb form and submit **only** this file. It must be named **CSE447_Assignment6.ipynb** in order for the autograder to work.
- At a low-load time on Gradescope, the autograder for this assignment takes about **two minutes** to run. We recommend submitting the code part of your assignment with enough time before the deadline to allow the autograder to run, and for you to investigate/correct any issues it flags.

## Section 0: Setup

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
%%bash

# get any other necessary files for this project
if [ ! -e "cache/" ]; then
  mkdir cache
fi

if [ ! -e "data-needed.txt" ]; then
  if [ ! -e "data_path_to_download_url.py" ]; then
    wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/data_path_to_download_url.py
  else
    echo "data_path_to_download_url.py script already downloaded to runtime"
  fi

  wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/assignments/NLGandDistillation/WithTextSummarization/data-needed.txt

  # download all data files needed for the student-release version of this project (i.e., no hidden test files)
  DATA_NEEDED_FILE="data-needed.txt"
  closing_slash="/"
  while IFS= read -r line; do
    line="$(echo -e "${line}" | sed -e 's/^[[:space:]]*//' -e 's/[[:space:]]*$//')";
    dirs_to_check="${line%${closing_slash}*}"
    mkdir -p $dirs_to_check
    download_url=$(python data_path_to_download_url.py "$line")
    echo $download_url;
    wget "$download_url" -O "$line"
  done < "$DATA_NEEDED_FILE"
else
  echo "data-needed.txt (and presumably therefore all necessary data files) already downloaded to runtime"
fi

if [ ! -e "other-setup-needed.sh" ]; then
  wget https://raw.githubusercontent.com/serrano-s/NLPassignments-students/refs/heads/main/assignments/NLGandDistillation/WithTextSummarization/other-setup-needed.sh
  bash other-setup-needed.sh
  rm data_path_to_download_url.py
else
  echo "other-setup-needed.sh (and presumably therefore all other necessary files) already downloaded to runtime"
fi

In [ ]:
%%bash
pip install transformers==4.57.1 sentencepiece==0.2.1 protobuf==5.29.5 datasets==4.0.0 evaluate==0.4.6

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

In [ ]:
"""set device and random seeds"""

######################################################
#  The following helper functions are given to you.
######################################################
import os
import torch
import torch.nn.functional as F
import numpy as np
import random
# Don't change the next four lines of code, the autograder needs them as-is
if __name__ == '__main__':
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = device
print(f"device: {device}")


def set_seed(seed=19260817):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed()

### 0.1 Dataset

In [ ]:
"""load datasets"""

######################################################
#  The following helper code is given to you.
######################################################

if __name__ == '__main__':
    from datasets import load_dataset

    dataset = load_dataset("Ximing/ROCStories")
    train_data, dev_data, test_data = (
        dataset["train"],
        dataset["validation"],
        dataset["test"],
    )

    print(train_data[0])

### 0.2 Evaluation Metrics

In [ ]:
"""prepare evaluation"""

######################################################
#  The following helper code is given to you.
######################################################

if __name__ == '__main__':
    from evaluate import load
    from transformers import RobertaForSequenceClassification, RobertaTokenizer

    perplexity_scorer = load("perplexity", module_type="metric")
    cola_model_name = "textattack/roberta-base-CoLA"
    cola_tokenizer = RobertaTokenizer.from_pretrained(cola_model_name)
    cola_model = RobertaForSequenceClassification.from_pretrained(cola_model_name).to(
        device
    )

def batchify(data, batch_size):
    assert batch_size > 0

    batch = []
    for item in data:
        # Yield next batch
        if len(batch) == batch_size:
            yield batch
            batch = []

        batch.append(item)

    # Yield last un-filled batch
    if len(batch) != 0:
        yield batch

In [ ]:
"""set up evaluation metric"""

######################################################
#  The following helper code is given to you.
######################################################


def compute_perplexity(texts, model="gpt2", batch_size=8):
    score = perplexity_scorer.compute(
        predictions=texts, add_start_token=True, batch_size=batch_size, model_id=model
    )
    return score["mean_perplexity"]


def compute_fluency(texts, batch_size=8):
    scores = []
    for b_texts in batchify(texts, batch_size):
        inputs = cola_tokenizer(
            texts, padding=True, truncation=True, return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            logits = cola_model(**inputs).logits
            probs = logits.softmax(dim=-1)
            scores.extend(probs[:, 1].tolist())
    return sum(scores) / len(scores)


def compute_diversity(texts):
    unigrams, bigrams, trigrams = [], [], []
    total_words = 0
    for gen in texts:
        o = gen.split(" ")
        total_words += len(o)
        for i in range(len(o)):
            unigrams.append(o[i])
        for i in range(len(o) - 1):
            bigrams.append(o[i] + "_" + o[i + 1])
        for i in range(len(o) - 2):
            trigrams.append(o[i] + "_" + o[i + 1] + "_" + o[i + 2])
    return (
        len(set(unigrams)) / len(unigrams),
        len(set(bigrams)) / len(bigrams),
        len(set(trigrams)) / len(trigrams),
    )


def evaluate(generations, experiment):
    generations = [_ for _ in generations if _ != ""]
    perplexity = compute_perplexity(generations)
    fluency = compute_fluency(generations)
    diversity = compute_diversity(generations)
    print(experiment)
    print(f"perplexity = {perplexity:.2f}")
    print(f"fluency = {fluency:.2f}")
    print(f"diversity = {diversity[0]:.2f}, {diversity[1]:.2f}, {diversity[2]:.2f}")
    print()


debug_sents = [
    "This restaurant is awesome",
    "My dog is cute and I love it.",
    "Today is sunny.",
]
if __name__ == '__main__':
    evaluate(debug_sents, "debugging run")

### 0.3: Load Model

In [ ]:
"""load model and tokenizer"""

######################################################
#  The following helper code is given to you.
######################################################

from transformers import GPT2LMHeadModel, GPT2Tokenizer, DynamicCache

if __name__ == '__main__':
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name, pad_token="<|endoftext|>")
    model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
    model.eval()

## **Section 1: Decoding Algorithms**

In this section, you will implement a few basic decoding algorithms:
1. Greedy decoding
2. Vanilla sampling
3. Temperature sampling
4. Top-k sampling
5. Top-p sampling

We have provided a wrapper function `decode()` that takes care of batching, controlling max length, and handling the EOS token.
You will be asked to implement the core function of each method: *given the pre-softmax logits of the next token, decide what the next token is.*

**The wrapper calls the core function of each decoding algorithm, which you will implement in the subsections below.**

In [ ]:
"""decode main wrapper function"""

######################################################
#  The following helper code is given to you.
######################################################

def _update_model_kwargs_for_generation(
    outputs, model_kwargs, is_encoder_decoder: bool = False
):
    # update past
    if "past_key_values" in outputs:
        model_kwargs["past_key_values"] = outputs.past_key_values
    elif "mems" in outputs:
        model_kwargs["past_key_values"] = outputs.mems
    elif "past_buckets_states" in outputs:
        model_kwargs["past_key_values"] = outputs.past_buckets_states
    else:
        model_kwargs["past_key_values"] = None

    # update token_type_ids with last value
    if "token_type_ids" in model_kwargs:
        token_type_ids = model_kwargs["token_type_ids"]
        model_kwargs["token_type_ids"] = torch.cat([token_type_ids, token_type_ids[:, -1].unsqueeze(-1)], dim=-1)

    # update attention mask
    if not is_encoder_decoder:
        if "attention_mask" in model_kwargs:
            attention_mask = model_kwargs["attention_mask"]
            model_kwargs["attention_mask"] = torch.cat(
                [attention_mask, attention_mask.new_ones((attention_mask.shape[0], 1))], dim=-1
            )

    return model_kwargs

def decode(prompts, max_len, method, **kwargs):
    encodings_dict = tokenizer(prompts, return_tensors="pt", padding=True)
    input_ids = encodings_dict["input_ids"].to(device)
    attention_mask = encodings_dict["attention_mask"].to(device)

    cache_position = torch.arange(input_ids.shape[1], dtype=torch.int64, device=device)

    model_kwargs = {"attention_mask": attention_mask}
    batch_size, input_seq_len = input_ids.shape
    past_key_values = DynamicCache()

    unfinished_sequences = torch.ones(batch_size, dtype=torch.long, device=device)

    for step in range(max_len):
        dict_of_inputs = {'past_key_values': past_key_values}
        dict_of_inputs.update(model_kwargs)
        if 'cache_position' not in dict_of_inputs or dict_of_inputs['cache_position'] is None:
            dict_of_inputs['cache_position'] = cache_position
        model_inputs = model.prepare_inputs_for_generation(input_ids, **dict_of_inputs)

        if 'cache_position' in model_inputs and model_inputs['cache_position'] is not None:
            dict_of_inputs = {'past_key_values': past_key_values}
            dict_of_inputs.update(model_inputs)
            dict_of_inputs['return_dict'] = True
            dict_of_inputs['output_attentions'] = False
            dict_of_inputs['output_hidden_states'] = False
            dict_of_inputs['use_cache'] = True
            with torch.no_grad():
                outputs = model(
                    **dict_of_inputs
                )
        else:
            dict_of_inputs = {'past_key_values': past_key_values}
            dict_of_inputs.update(model_inputs)
            dict_of_inputs['return_dict'] = True
            dict_of_inputs['output_attentions'] = False
            dict_of_inputs['output_hidden_states'] = False
            dict_of_inputs['use_cache'] = True
            dict_of_inputs['cache_position'] = cache_position
            with torch.no_grad():
                outputs = model(
                    **dict_of_inputs
                )

        if step == 0:
            last_non_masked_idx = torch.sum(attention_mask, dim=1) - 1
            next_token_logits = outputs.logits[
                range(batch_size), last_non_masked_idx, :
            ]
        else:
            next_token_logits = outputs.logits[:, -1, :]

        log_prob = F.log_softmax(next_token_logits, dim=-1)

        if method == "greedy":
            next_tokens = greedy(next_token_logits)
        elif method == "sample":
            next_tokens = sample(next_token_logits)
        elif method == "temperature":
            next_tokens = temperature(next_token_logits, temperature=kwargs.get("temperature", 0.8))
        elif method == "topk":
            next_tokens = topk(
                next_token_logits,
                k=kwargs.get("k", 20),
                temperature=kwargs.get("temperature", 1.0),
            )
        elif method == "topp":
            next_tokens = topp(
                next_token_logits,
                p=kwargs.get("p", 0.7),
                temperature=kwargs.get("temperature", 1.0),
            )

        # finished sentences should have their next token be a padding token
        next_tokens = next_tokens * unfinished_sequences + tokenizer.pad_token_id * (
            1 - unfinished_sequences
        )

        input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)

        model_kwargs = _update_model_kwargs_for_generation(
            outputs, model_kwargs, is_encoder_decoder=model.config.is_encoder_decoder
        )

        # if eos_token was found in one sentence, set sentence to finished
        unfinished_sequences = unfinished_sequences.mul(
            (next_tokens != tokenizer.eos_token_id).long()
        )

        cache_position = cache_position[-1:] + 1

        if unfinished_sequences.max() == 0:
            break

    response_ids = input_ids[:, input_seq_len:]
    response_text = [
        tokenizer.decode(
            output, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        for output in response_ids
    ]

    return response_text

In [ ]:
"""debug helper code"""

######################################################
#  The following helper code is given to you.
######################################################

if __name__ == '__main__':
    # For debugging, we duplicate a single prompt 10 times so that we obtain 10 generations for the same prompt
    dev_prompts = [dev_data[0]["prompt"]] * 10

def print_generations(prompts, generations):
    for prompt, generation in zip(prompts, generations):
        print(f"{[prompt]} ==> {[generation]}")

### 1.1: Greedy Decoding

In [ ]:
def greedy(next_token_logits):
    """
    Applies greedy decoding to get the next token.
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)

    """
    next_tokens = None

    raise NotImplementedError
    return next_tokens

In [ ]:
if __name__ == '__main__':
    generations = decode(dev_prompts, max_len=20, method="greedy")
    print_generations(dev_prompts, generations)

### 1.2: Vanilla Sampling and Temperature Sampling

All our remaining decoding techniques involve nondeterminism-- that is, sampling. Your implementations of the remaining four decoding techniques in this assignment should all call the following function that we provide to you, `sample_next_tokens_from_probability_distribution`:

In [ ]:
def sample_next_tokens_from_probability_distribution(probs):
    """
    inputs:
    - probs: Tensor(size = (B, V), dtype = float)
      probs should be a tensor that contains one probability distribution per
      instance, with each instance's probability distribution over the full
      set of vocabulary words.
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
      next_tokens contains one token index per instance of text, corresponding
      to the sampled next instance.
    """
    next_tokens = torch.tensor([torch.multinomial(probs[i], num_samples=1) for i in range(probs.shape[0])],
                                device=DEVICE)
    return next_tokens

**Note that in order for your code to work with our autograder, your implementations of the remaining decoding techniques MUST each call `sample_next_tokens_from_probability_distribution`.**

With that said, let's start by implementing vanilla sampling:

In [ ]:
def sample(next_token_logits):
    """
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
    """

    probs = None


    raise NotImplementedError

    # Sample from your calculated probabilities -- DON'T CHANGE THE REMAINING CODE OF THIS FUNCTION,
    # it's important for autograding
    next_tokens = sample_next_tokens_from_probability_distribution(probs)

    return next_tokens

In [ ]:
if __name__ == '__main__':
    set_seed()
    generations = decode(dev_prompts, max_len=20, method="sample")
    print_generations(dev_prompts, generations)

In [ ]:
def temperature(next_token_logits, temperature):
    """
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    - temperature: Temperature parameter float
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
    """
    probs = None


    raise NotImplementedError

    # Sample from your calculated probabilities -- DON'T CHANGE THE REMAINING CODE OF THIS FUNCTION,
    # it's important for autograding
    next_tokens = sample_next_tokens_from_probability_distribution(probs)

    return next_tokens

In [ ]:
if __name__ == '__main__':
    set_seed()
    generations = decode(dev_prompts, max_len=20, method="temperature", temperature=0.8)
    print_generations(dev_prompts, generations)

One way to check your implementation is to see if you get same results as greedy decoding when temperature to be a very small value like 0.001 and same results as vanilla sampling when temperature = 1.

### 1.3: Top-k Sampling

Useful tips:
- Recall that in Top-k sampling, we only sample from the top-k tokens with the highest probabilities. To ensure that we set the logits other than the top-k to be -inf. You can use `float("-inf")` to represent infinity in python.
- You will find `torch.topk()` useful for getting the top-k logits and indices. Check out the [documentation](https://pytorch.org/docs/stable/generated/torch.topk.html) for the function for more details.
- Do not forget to divide the logits by the temperature before applying softmax.

In [ ]:
def topk(next_token_logits, k, temperature = 1):

    """
    Applies the top-k sampling decoding algorithm to get the next token.
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    - k: int, the number of top tokens to consider
    - temperature: Temperature parameter float
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
    """
    probs = None

    raise NotImplementedError

    # Sample from your calculated probabilities -- DON'T CHANGE THE REMAINING CODE OF THIS FUNCTION,
    # it's important for autograding
    next_tokens = sample_next_tokens_from_probability_distribution(probs)

    return next_tokens

In [ ]:
if __name__ == '__main__':
    set_seed()
    generations = decode(dev_prompts, max_len=20, method="topk", k=20)
    print_generations(dev_prompts, generations)

One way to check your implementation is to see if you get same results as greedy decoding when k = 1 and same results as vanilla sampling when k = V i.e. 50257.

### 1.4: Top-p Sampling

In [ ]:
def topp(next_token_logits, p, temperature = 1):
    """
    Applies the top-p sampling or nucleus sampling decoding algorithm to get the next token.
    inputs:
    - next_token_logits: Tensor(size = (B, V), dtype = float)
    - p: float, the cutoff probability for the top-p sampling
    - temperature: Temperature parameter float
    outputs:
    - next_tokens: Tensor(size = (B), dtype = long)
    """

    # TODO: Sort the logits in descending order, and compute
    # the cumulative probabilities `cum_probs` on the sorted logits
    sorted_logits, sorted_indices = None, None
    sorted_probs = None
    cum_probs = None

    raise NotImplementedError

    # Create a mask to zero out all logits not in top-p
    sorted_indices_to_remove = cum_probs > p
    sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
    sorted_indices_to_remove[:, 0] = 0
    # Restore mask to original indices
    indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)

    # indices_to_remove now contains a two-dimensional mask (instance_index by possible_next_token_index)
    # with 0s wherever a token's probability was part of that instance's top p probability mass, and
    # 1s wherever a token's probability was low enough for it to be knocked out of consideration for
    # the actual sampling to follow.

    probs = None
    # TODO: populate probs with the final adjusted version of next-token probabilities (a two-dimensional
    # tensor with one vector of next-token probabilities per instance). You'll need to use the previously calculated
    # indices_to_remove mask.

    raise NotImplementedError

    # Sample from your calculated probabilities -- DON'T CHANGE THE REMAINING CODE OF THIS FUNCTION,
    # it's important for autograding
    next_tokens = sample_next_tokens_from_probability_distribution(probs)

    return next_tokens

In [ ]:
if __name__ == '__main__':
    set_seed()
    generations = decode(dev_prompts, max_len=20, method="topp", p=0.7)
    print_generations(dev_prompts, generations)

To check your implementation, you can see if you get same results as greedy decoding when p = 0 and same results as vanilla sampling when p = 1.

### 1.5: Evaluate!

Run the following cell to obtain the evaluation results, which you should include in your writeup.
Also don't forget to answer the questions.

**Note that on a (default) CPU runtime, the following cell takes about an hour to run**, while on a T4 GPU runtime, it takes less than two minutes. **So if available, and if you haven't switched already, we'd recommend changing your runtime type to one using a GPU** (Runtime > Change runtime type, and then select any GPU-based runtime).

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
print('Results generated on runtime with device', str(device))

set_seed()
prompts = [item["prompt"] for item in test_data][:10]
GENERATIONS_PER_PROMPT = 10
MAX_LEN = 100

for experiment in ["greedy", "sample", "temperature", "topk", "topp"]:
    generations = []
    for prompt in tqdm(prompts):
        generations += decode(
            [prompt] * GENERATIONS_PER_PROMPT, max_len=MAX_LEN, method=experiment
        )
    evaluate(generations, experiment)

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

You should see the following values:

- For greedy:
**perplexity** = 2.08 /
**fluency** = 0.78 /
**diversity** = 0.01, 0.02, 0.03

- For sample:
**perplexity** = 69.23 on CPU, 63.20 on a T4 GPU /
**fluency** = 0.35 on CPU, 0.39 on a T4 GPU /
**diversity** = 0.42, 0.90, 1.00 on CPU; 0.44, 0.90, 0.99 on a T4 GPU

- For temperature:
**perplexity** = 15.15 on CPU, 16.00 on a T4 GPU /
**fluency** = 0.63 on CPU, 0.62 on a T4 GPU /
**diversity** = 0.32, 0.78, 0.95 on CPU; 0.31, 0.79, 0.96 on a T4 GPU

- For topk:
**perplexity** = 12.53 on CPU, 13.39 on a T4 GPU /
**fluency** = 0.71 on CPU, 0.68 on a T4 GPU /
**diversity** = 0.27, 0.74, 0.96 on CPU; 0.25, 0.74, 0.95 on a T4 GPU

- For topp:
**perplexity** = 12.26 on CPU, 12.68 on a T4 GPU /
**fluency** = 0.71 on CPU, 0.72 on a T4 GPU /
**diversity** = 0.28, 0.75, 0.95 on CPU; 0.29, 0.76, 0.95 on a T4 GPU

### *Do I always need to use all this code to generate text from a language model?*

The exercises above were to help you understand the underlying mechanisms of different decoding methods. In practice, you don't need to implement all these decoding methods from scratch. You can use the `generate()` method in the 🤗 Transformers library to generate text from a language model. Below we provide an example of how to use the `generate()` method to generate text from a language model. Please pay close attention to this especially if you are going to be attempting the optional section (mandatory for 517 students) on Knowledge Distillation.

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Step 0: Load a language model, and it's the corresponding tokenizer from the Hugging Face model hub

model = AutoModelForCausalLM.from_pretrained("gpt2", cache_dir="cache/")
tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side="left", cache_dir="cache/") # Setting padding_side=left is important when using these models for open ended text generation

# Move the model to the device
model = model.to(device)

example_prompt = "Once upon a time"

# Step 1: Tokenize the input prompt
tokenized_input = tokenizer(example_prompt, return_tensors="pt").to(device)
print("Tokenizer output:")
print(tokenized_input)
print("*******************")

# Step 2: Generate text from the model
output = model.generate(**tokenized_input, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=1.0,)
print("Model Generate output:")
print(output)
print("*******************")

# Step 3: Convert the output ids to text
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Text output:")
print(output_text)
print("*******************")
# Step 3 (Optional): .generate() returns the model inputs as well. We can ignore that by slicing the output
output_text = tokenizer.decode(output[0][len(tokenized_input.input_ids[0]):], skip_special_tokens=True)
print("Text output (ignoring model inputs):")
print(output_text)
print("*******************")

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

Let's go over each of these one by one:

- **Step 1:** The `tokenizer()` function takes the input prompt and converts it into a format that the model can understand. This is essentially converting the input prompt into a sequence of tokens. Note that the tokenizer returns a dictionary with the tokenized input ids and attention mask. The `input_ids` are the token ids that the model will use as input. `"attention_mask`" is a mask vector that indicates if a particular token corresponds to padding. Padding is extremely important when we are dealing with variable length sequences. Through padding, we can ensure that all the sequences in a batch are of the same size. When feeding sequences with padding to a transformer based model, we need to make sure that the model doesn't attend to the padding tokens. The `attention_mask` is used the tokens that are to be ignored in the attention operation. Note that here since we only used a single input sequence, there was no need of padding and that's why all the attention mask values are 1, i.e. none of the tokens in the sequences should be ignored by attention blocks.

- **Step 2:** The `generate()` function takes the tokenized input and generates a sequence of tokens as output. The function takes in a number of arguments, the most important of which are `max_new_tokens` and `do_sample`. The `max_new_tokens` argument specifies the maximum number of new tokens to be generated. The `do_sample` argument is used to turn on sampling, if false, greedy decoding is used. If `do_sample` is set to `True`, then the `top_p` and `temperature` arguments are used to control the sampling process. The `top_p` argument is the value of $p$ parameter for top-p or nucleus sampling and the `temperature` argument is used to control the temperature sampling. Notice that the `generate()` function returns as output a sequence of token ids.

- **Step 3:** The `tokenizer.decode` method converts the token ids into text. The `skip_special_tokens=True` argument is used to ignore the special tokens (e.g. start of sequence, end of sequence, padding etc.) that are added by the tokenizer.

- **Step 3 (Optional):** The `generate()` function returns the model inputs as well. We can ignore that by slicing the output.

## [Optional for CSE 447]  **Section 2: Knowledge Distillation**

In this part of the homework, we will learn how we can use knowledge distillation from a larger teacher model to a smaller student model. Particularly, we will be focusing on the task of text summarization and using the CNN/Daily Mail dataset. We will use Qwen2.5-1.5B-Instruct as our teacher model, which is a 1.5B parameter decoder-only mode pre-trained on 18T tokens of data and then further fine-tuned to follow instructions to perform different tasks (similar to something like ChatGPT). You can read more about Qwen2.5 models [here](https://qwenlm.github.io/blog/qwen2.5/). For the student model, we will be using the default GPT-2 model, which is a 124M parameter model.

Since Qwen2.5-1.5B-Instruct is a much bigger model and trained on a lot of data, it is more capable of generating better summaries as compared to the default GPT-2 model which is a smaller model and has seen less data. Knowledge distillation is a technique to transfer the knowledge of a larger teacher model to a smaller student model. In this way, we can leverage the large amount of data and compute resources used to train the teacher model to improve the performance of the student model.

This assignment will also make heavy use of the [🤗 Transformers Library](https://huggingface.co/docs/transformers/index). Don't worry if you are not familiar with the library, we will discuss its usage in detail.

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
%%bash
pip install rouge-score==0.1.2
pip install evaluate==0.4.6

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

In [ ]:
# Load packages for this section
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
if __name__ == '__main__':
    from pprint import pprint
    from datasets import load_from_disk, Dataset, DatasetDict
    from tqdm import tqdm


As always, we will start by loading the dataset.

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
######################################################
#  The following code is given to you. DO NOT MODIFY.
######################################################
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, "data")
cnn_dm_cse447_dataset = load_from_disk(os.path.join(data_dir, "cnn_dm_cse447_dataset.hf"))
cnn_dm_cse447_dataset

In [ ]:
# Preview the dataset
print("Full article:")
pprint(cnn_dm_cse447_dataset["val"][0]["article"])
print("\n\n")
print("Gold summary:")
pprint(cnn_dm_cse447_dataset["val"][0]["summary"])

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

### 2.1: Student Model

In this exercise we will try to understand how well the student model i.e. GPT-2 does on the summarizaion task out of box. In later exercises we will try to improve the performance of the student model using knowledge distillation.

We start by loading the student model and tokenizer.

In [ ]:
######################################################
#  The following code is given to you. DO NOT MODIFY.
######################################################
from transformers import AutoModelForCausalLM, AutoTokenizer

if __name__ == '__main__':
    student_model = AutoModelForCausalLM.from_pretrained("gpt2")
    student_tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side="left")

    student_model = student_model.to(device)
    student_tokenizer.pad_token_id = student_tokenizer.eos_token_id
    student_model

Notice GPT-2 is a decoder-only model and has 12 layers. This is the smallest model in the GPT-2 family i.e. with only 124M parameters.

### 2.1.1 Preparing Data for Student Model

Our student model is a language model and inherently a language model's job is to predict continuations of a sequence by predicting one token at a time. To perform specific tasks like summarization using language models, we need to prepare the data in such a format such that the possible continuation of the sequence is the output we want i.e. in this case the summary of the article.

The GPT-2 paper found adding a TL;DR to the end of the article helps the model in generating better summaries. Implement the `prepare_articles` function that adds a TL;DR: to the end of each input articles and then tokenizes the articles.

Useful tips:
- While tokenizing the articles, by calling `tokenizer()` make sure to set `padding="max_length"` and `truncation=True` so that the articles are padded to the same length and truncated if they are longer than the maximum length. Also, make sure to set `return_tensors="pt"` so that the output is a PyTorch tensor and `max_length` to the maximum length of the articles.

In [ ]:
def prepare_articles_for_student_model(articles, student_tokenizer, max_len=1024):

    """
    Processes and tokenizes articles into a format that can be used for summarization by the student model
    and then tokenizes the articles using the student tokenizer.

    Inputs:
    - articles: A list of articles to be summarized.
    - student_tokenizer: The tokenizer to use for tokenizing the articles.
    - max_len: The maximum length of the articles.

    Returns:
    - tokenized_articles: A dictionary containing the input ids and attention mask of the tokenized articles.
    """
    tokenized_articles = None


    raise NotImplementedError
    return tokenized_articles

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
######################################################
#  The following code is given to you. DO NOT MODIFY.
######################################################


def test_prepare_articles_for_student_model():
    # Setup
    student_tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side="left")
    student_tokenizer.pad_token_id = student_tokenizer.eos_token_id

    # Test 1: Basic functionality
    articles = ["This is a test article."]
    output = prepare_articles_for_student_model(articles, student_tokenizer)
    assert "input_ids" in output, "Output should contain input_ids"
    assert "attention_mask" in output, "Output should contain attention_mask"
    assert torch.is_tensor(output["input_ids"]), "input_ids should be a tensor"
    assert torch.is_tensor(
        output["attention_mask"]
    ), "attention_mask should be a tensor"

    # Test 2: Multiple articles
    articles = ["First article.", "Second article.", "Third article."]
    output = prepare_articles_for_student_model(articles, student_tokenizer)
    assert (
        output["input_ids"].shape[0] == 3
    ), "Batch size should match number of articles"
    assert (
        output["attention_mask"].shape[0] == 3
    ), "Batch size should match number of articles"

    # Test 3: TL;DR addition
    articles = ["Test article."]
    output = prepare_articles_for_student_model(articles, student_tokenizer)
    decoded = student_tokenizer.decode(output["input_ids"][0], skip_special_tokens=True)
    assert "TL;DR:" in decoded, "TL;DR: should be added to the article"
    assert decoded == "Test article.\nTL;DR:", "Article format should be correct"

    # Test 4: Padding and truncation
    long_article = "This is a very " * 1000  # Create a very long article
    short_article = "Short."
    articles = [long_article, short_article]
    output = prepare_articles_for_student_model(articles, student_tokenizer)
    assert (
        output["input_ids"].shape[1] == 1024
    ), "Should be padded/truncated to max length"
    assert (output["attention_mask"][1] == 0).any(), "Short article should have padding"

    print("All tests passed!")

test_prepare_articles_for_student_model()
######################################################

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

### 2.1.2 Summarizing with Student Model

Use the `generate()` method to generate summaries from the student model. Refer to the end of the Section 1 of the notebook for an example of how to use the `generate()` method. You can also learn more about the `generate()` method [here](https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationMixin.generate).

Helpful tips:
- Instead of generating summaries one by one, we recommend generating summaries in batches to speed up the process.

In [ ]:
def summarize_wth_student_model(
    articles,
    student_model,
    student_tokenizer,
    batch_size=8,
    max_new_tokens=100,
    do_sample=True,
    p=0.9,
    temperature=1.0,
    device=device
):

    """
    Generates a list of summaries for a list of articles using the student model.

    Inputs:
    - articles: A list of articles to be summarized.
    - student_model: The student model to use for summarization.
    - student_tokenizer: The tokenizer corresponding to the student model.
    - batch_size: The batch size to use for summarization.
    - max_new_tokens: The maximum number of new tokens to generate.
    - do_sample: Whether to use sampling or greedy decoding.
    - p: The p parameter for top-p sampling.
    - temperature: The temperature for sampling.

    Returns:
    - summaries: A list of summaries for the articles.
    """

    summaries = []
    student_model.eval()

    for i in tqdm(range(0, len(articles), batch_size)):
        # ToDo: Tokenize the batch of articles

        raise NotImplementedError
        # Move the data to the device
        tokenized_articles = {
            key: value.to(device) for key, value in tokenized_articles.items()
        }

        # ToDo: Generate summaries from the model using the .generate method

        raise NotImplementedError

        # ToDo: Convert the generated summaries to text. Hint: Use the `batch_decode` method of the tokenizer.
        # Make sure to only decode the generated tokens, i.e., ignore the input tokens.

        raise NotImplementedError

        # Add the decoded summaries to the list
        summaries.extend(decoded_summaries)

    return summaries

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
def test_summarize_wth_student_model():
    # Test 1: Basic functionality
    articles = ["This is a test article about AI."]
    summaries = summarize_wth_student_model(
        articles, student_model, student_tokenizer, device=device
    )
    assert isinstance(summaries, list), "Output should be a list"
    assert len(summaries) == len(articles), "Should generate one summary per article"
    assert isinstance(summaries[0], str), "Each summary should be a string"
    assert len(summaries[0]) > 0, "Summaries should not be empty"

    # Test 2: Batch processing
    articles = [
        "First article.",
        "Second article.",
        "Third article.",
    ] * 4  # 12 articles
    batch_size = 4
    summaries = summarize_wth_student_model(
        articles, student_model, student_tokenizer, batch_size=batch_size, device=device
    )
    assert len(summaries) == len(articles), "Should generate summary for each article"

    # Test 3: Generation parameters
    articles = [
        "Test article for parameter checking.",
        "Test article for parameter checking.",
    ]
    # Test with different generation parameters
    summaries_greedy = summarize_wth_student_model(
        articles, student_model, student_tokenizer, do_sample=False, device=device
    )
    summaries_sampling = summarize_wth_student_model(
        articles,
        student_model,
        student_tokenizer,
        do_sample=True,
        p=0.9,
        temperature=0.7,
        device=device,
    )

    assert summaries_greedy[0] == summaries_greedy[1], "Greedy decoding should give the same result"
    assert (
        summaries_sampling[0] != summaries_sampling[1]
    ), "Sampling should give different results"

    # Test 4: Check only generated tokens are in summary
    article = "This is a test article about artificial intelligence. It contains specific phrases that should not appear in the summary unless generated."
    summaries = summarize_wth_student_model(
        [article], student_model, student_tokenizer, device=device
    )
    # Verify the input article text isn't in the summary
    assert article not in summaries[0], "Summary should not contain the input article"

    print("\nAll tests passed!")

test_summarize_wth_student_model()

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

Let's generate summaries for the validation set using the student model.

**If you're on a fully CPU-based runtime, this next cell will take around four hours to run.** On a T4 GPU-based runtime, it takes around seven minutes. **We'd highly recommend switching to a GPU-based runtime** (and making sure that the `device` variable, defined up near the top of this notebook, is updated) **before running this cell** (Runtime > Change runtime type).

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
set_seed()
pred_summaries = summarize_wth_student_model(
    cnn_dm_cse447_dataset["val"]["article"],
    student_model,
    student_tokenizer,
    p=0.9,
    temperature=1.0,
    device=device
)

In [ ]:
# Inspect the summaries
print("Article:")
pprint(cnn_dm_cse447_dataset["val"][0]["article"])
print("***********************\n\n")
print("Generated summary:")
pprint(pred_summaries[0])
print("***********************\n\n")
print("Gold summary:")
pprint(cnn_dm_cse447_dataset["val"][0]["summary"])


In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

As you can see, the generated summary is not very good, with a lot of completely irrelevant text. Let's try to quantify this by evaluating the summaries using the ROUGE score.

### Evaluating Student Model

We will use the ROUGE score to evaluate the summaries generated by the student model. ROUGE measures how similar a generated summary is to a reference (human-written) summary by comparing overlapping words and phrases. Think of it like a sophisticated "spot the differences" between texts.

Example:
- **Reference**: "The cat sat on the mat"
- **Generated**: "The cat lay on the mat"

ROUGE has three main variants:
1. **ROUGE-1**: Matches single words (In example: 5/6 words match)
2. **ROUGE-2**: Matches word pairs (In example: "the cat", "on the", "the mat" match)
3. **ROUGE-L**: Finds longest matching sequences in order

Scores range from 0 to 1, with higher being better. While widely used in summarization evaluation, ROUGE isn't perfect - it focuses on matching words rather than meaning.

Reference: Lin, C. Y. (2004). ROUGE: A package for automatic evaluation of summaries.

In [ ]:
# Evaluate the summaries
if __name__ == '__main__':
    import evaluate
    rouge = evaluate.load("rouge")
    scores = rouge.compute(predictions=pred_summaries, references=cnn_dm_cse447_dataset["val"]["summary"], use_stemmer=True)
    print(scores)

You should see the following scores:
- rouge1: 0.19
- rouge2: 0.03
- rougeL: 0.13
- rougeLsum: 0.16


The model performs relatively poorly on the dataset, especially considering the ROUGE-2 and ROUGE-L scores, which are very low. Let's see if we can improve this by using a teacher model.

### 2.2 Generate Data from a Teacher Model

We will now start with step 1 of knowledge distillation i.e. generating data from a teacher model. As we mentioned earlier, we will use the Qwen2.5-1.5B-Instruct model as our teacher model. Note that Qwen2.5-1.5B-Instruct is an instruction tuned model, which is different from a base language model like GPT-2. Instruction tuned models are obtained from base language models by fine-tuning them on a diverse set of instructions and their desired outputs. This enables the model to become better at following instructions and is a big factor in the success of recent large language models.

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
# Load a teacher model.
from transformers import AutoModelForCausalLM, AutoTokenizer
teacher_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct", cache_dir="cache/"
)
teacher_tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct", cache_dir="cache/", padding_side="left"
)
teacher_model = teacher_model.to(device)
teacher_model

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

### 2.2.1 Preparing Data for Teacher Model

We need to prepare the data for the teacher model in a specific format. Since the teacher model is an instruction-tuned model, it has been adapted to follow instructions. Hence, instead of formatting the text according to a completion problem, we need to format it according to an instruction following the problem. We will do this by adding an instruction to the beginning of each article, i.e., "Summarize the following article." Further, we will also instruct the model to output the summary in a specific format by appending a suffix to the end of each article, i.e., "Start your summary with 'TL;DR:'" In short, the articles should be formatted as follows:

`"Summarize the following article: <article>. Start your summary with 'TL;DR:'"`

**To match our implementation, you need to *exactly* match the above format.**

In [ ]:
def prepare_articles_teacher(articles, teacher_tokenizer, max_len=1024):

    """
    Prepares the articles for the teacher model and performs tokenization.

    Inputs
    """

    # ToDo: Add the instructions as specified above to each article
    articles_with_instructions = None


    raise NotImplementedError

    # The instruction-tuned models to be in chat format. Read more here: https://huggingface.co/docs/transformers/main/en/chat_templating
    articles_chat = [[
        {"role": "system", "content": "You are a helpful assistant and an expert at summarizing articles."},
        {"role": "user", "content": article}
    ] for article in articles_with_instructions]
    # Apply chat template to the articles
    articles_chat = teacher_tokenizer.apply_chat_template(articles_chat, tokenize=False, add_generation_prompt=True)

    # ToDo: Tokenize the articles. Hint: Set `add_special_tokens=False` since special tokens are already added in the chat template.


    raise NotImplementedError

    return tokenized_articles

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
def test_prepare_articles_teacher():

    # Test 1: Basic functionality
    articles = ["This is a test article."]
    output = prepare_articles_teacher(articles, teacher_tokenizer)

    assert "input_ids" in output, "Output should contain input_ids"
    assert "attention_mask" in output, "Output should contain attention_mask"
    assert torch.is_tensor(output["input_ids"]), "input_ids should be a tensor"
    assert torch.is_tensor(
        output["attention_mask"]
    ), "attention_mask should be a tensor"

    # Test 2: Instruction addition
    decoded = teacher_tokenizer.decode(output["input_ids"][0], skip_special_tokens=True)
    assert (
        decoded == f"system\nYou are a helpful assistant and an expert at summarizing articles.\nuser\nSummarize the following article: {articles[0]}. Start your summary with 'TL;DR:'\nassistant\n"
    ), "Should EXACTLY follow given summarization instruction formatting"

    # Test 3: Multiple articles
    articles = ["First article.", "Second article.", "Third article."]
    output = prepare_articles_teacher(articles, teacher_tokenizer)
    assert (
        output["input_ids"].shape[0] == 3
    ), "Batch size should match number of articles"
    assert (
        output["attention_mask"].shape[0] == 3
    ), "Batch size should match number of articles"

    # Test 4: Padding and truncation
    long_article = "This is a very " * 1000  # Create a very long article
    short_article = "Short."
    articles = [long_article, short_article]
    output = prepare_articles_teacher(articles, teacher_tokenizer, max_len=1024)
    assert (
        output["input_ids"].shape[1] == 1024
    ), "Should be padded/truncated to max length"
    assert (output["attention_mask"][1] == 0).any(), "Short article should have padding"

    print("All tests passed!")

test_prepare_articles_teacher()

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

Let's have a look at the tokenized articles.

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
tokenized_articles = prepare_articles_teacher(
    cnn_dm_cse447_dataset["val"]["article"][:2], teacher_tokenizer
)
decoded_articles = teacher_tokenizer.batch_decode(
    tokenized_articles["input_ids"], skip_special_tokens=True
)
pprint(decoded_articles[0])

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

Notice the system, user, and assistant tags in the prompt. The system tag is followed by an instruction to ground the model to be a helpful assistant and be an expert at summarizing articles. The user tag is followed by the article to be summarized and the instruction about the task. We have the assistant tag in the end and the model is expected to generate the summary after the assistant tag.

### 2.2.2. Generate summaries with teacher model

Similar to the student model, we will use the `generate()` method to generate summaries from the teacher model. Implement the `summarize_with_teacher_model` function below to do this.

In [ ]:
def summarize_with_teacher_model(
    articles,
    teacher_model,
    teacher_tokenizer,
    batch_size=8,
    max_new_tokens=100,
    do_sample=True,
    p=0.9,
    temperature=1.0,
    device=device,
):

    """
    Generates summaries for a list of articles using the teacher model.
    Essentially the same as the function `summarize_wth_student_model`, but we will use the prepare articles function for the teacher model here.
    """
    summaries = []
    teacher_model.eval()
    with torch.no_grad():
        for i in tqdm(range(0, len(articles), batch_size)):

            raise NotImplementedError
            # Add the decoded summaries to the list
            summaries.extend(decoded_summaries)

    return summaries

Let's first check how well the teacher model performs on the validation set.

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
set_seed()
# Since the teacher model is large, we will only evaluate on a small subset of the validation set.
val_summaries_generated = summarize_with_teacher_model(
    cnn_dm_cse447_dataset["val"]["article"][:100],
    teacher_model,
    teacher_tokenizer,
    batch_size=8,
    max_new_tokens=100,
    do_sample=True,
    p=0.9,
    temperature=1.0,
    device=device
)

In [ ]:
# Inspect the summaries
print("Article:")
pprint(cnn_dm_cse447_dataset["val"][0]["article"])
print("***********************\n\n")
print("Generated summary:")
pprint(val_summaries_generated[0])
print("***********************\n\n")
print("Gold summary:")
pprint(cnn_dm_cse447_dataset["val"][0]["summary"])

You should see that the generated summary is much better than the one generated by the student model. Let's now evaluate the summaries generated by the teacher model.

In [ ]:
set_seed()
teacher_rouge = evaluate.load("rouge")
scores = teacher_rouge.compute(
    predictions=val_summaries_generated,
    references=cnn_dm_cse447_dataset["val"]["summary"][:100],
    use_stemmer=True
)
print(scores)


In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

You should see the following scores:
- rouge1: 0.262
- rouge2: 0.082
- rougeL: 0.182
- rougeLsum: 0.211

While these are only on a small subset of the validation set, we also ran the teacher model on the entire validation set and got the following scores:
- rouge1: 0.284
- rouge2: 0.085
- rougeL: 0.188
- rougeLsum: 0.229

As you can see, the teacher model performs much better than the student model. While the numbers are not super high, note that our teacher model is only 1.5B parameters, which is considered a tiny model in the current LLM landscape. Bigger models with 8B, 30B, 70B, or even higher are expected to perform even better.

#### Generating Data for Distillation

We can now generate data for distillation. We will use the articles from the training dataset and generate summaries for those articles using the teacher model. Note that we do have the original summaries for the training set, but we are not using them for distillation. Our goal is to show you how distillation works and how you can use it for your own use cases where you might not have labelled training data available.

**Note that in our case, to save you time on this assignment, we've actually pre-generated this data;** you *can* run the next two blocks of code to re-generate the data, but they're mostly here so that you know exactly how the `kd_dataset.hf` data we provide you with was generated (and how you'd generate your own data for distillation in another setting). **For this assignment, though, you can skip ahead a couple of code blocks to the one defining `kd_dataset`.**

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
# Again, for this assignment, you can skip running this code block (and the one right after it)
# and instead run the one that just contains:
#
# kd_dataset = load_from_disk(f"{data_dir}/kd_dataset.hf")
#
# These two code blocks are only here to show you how the data for distillation was generated.
set_seed()
train_summaries_generated = summarize_with_teacher_model(
    cnn_dm_cse447_dataset["train"]["article"][:1000],
    teacher_model,
    teacher_tokenizer,
    batch_size=8,
    max_new_tokens=100,
    do_sample=True,
    p=0.9,
    temperature=1.0,
    device=device
)

In [ ]:
if not os.path.isfile(f"{data_dir}/kd_dataset.hf/data-00000-of-00001.arrow"):
    kd_dataset = {
        "article": cnn_dm_cse447_dataset["train"]["article"][:1000][:1000],
        "summary": train_summaries_generated,
        "gold_summary": cnn_dm_cse447_dataset["train"]["summary"][:1000][:1000]
    }
    kd_dataset = Dataset.from_dict(kd_dataset)
    kd_dataset

    # Save the dataset to disk, so that you can load it later without re-generating the data.
    kd_dataset.save_to_disk(f"{data_dir}/kd_dataset.hf")

For this assignment, you can directly load our previously teacher-model-generated data using the following cell:

In [ ]:
kd_dataset = load_from_disk(f"{data_dir}/kd_dataset.hf")

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

### 3. Fine-tuning student model with teacher model's generated data

#### 3.1 Preparing data for distillation

You will now implement the `prepare_data_for_distillation` function to prepare the data for distillation. This function will format the data in a specific way so that it can be used to fine-tune the student model. You will follow pretty much the same process as you did for the student model in `prepare_articles_for_student_model` with a few changes.

- First we will include the summaries in the input text along with the articles. This is done because we are now training the student model to generate summaries from the articles. Hence the format of the input text will be `<article>\nTL;DR:<summary>`.
- In the tokenization dictionary, we now need to add a new key, `labels,` which contains the labels to train the language model. For language models, the labels are the same as the input IDs since the model is expected to generate the next word in the sequence. However, while fine-tuning, we want the model to learn how to generate the summaries from the articles and we do not care about the model learning to predict tokens in the original articles. Therefore, we replace the labels for the prompt tokens with -100, which is a special token id that is used to signal the loss function to ignore the loss for those tokens.


In [ ]:
def prepare_data_for_distillation(article, summary, student_tokenizer, max_length=1024):

    """
    Prepares the data for distillation.

    Inputs:`
    - article: The article to be summarized.
    - summary: Summary to be used as labels for distillation.
    - student_tokenizer: Tokenizer for the student model.
    - max_length: Maximum length of the tokenized input.

    Returns:
    - tokenized_data: Tokenized data in the format of a dictionary with keys `input_ids`, `attention_mask`, and `labels`.

    Note: Unlike the functions preparing data for student and techer models before, this function only takes a single
    article as input instead of a list of articles. We are doing this because while training we want to create batches
    dynamically such that they are padded based on the longest article in the batch. Earlier functions were padding all
    the articles with the same length. While it doesn't make any difference in the performance whether you use
    dynamic padding or fixed padding, dynamic padding can be much more compute and memory efficient, which is especially
    important for training
    """

    # TODO: Format the article and summary for distillation.
    # Hint: `summary` might already contain the TL;DR: prefix, make sure you are not adding it twice!
    article_wth_summary = None
    article_prompt = None  # This should only contain the article and <article>\nTL;DR: you will need this to set the labels for the prompt tokens to -100


    raise NotImplementedError

    # TODO: Tokenize `articles_wth_summaries`. Note set padding=False, since we want to do dyanmic padding during training.
    tokenized_data = None
    tokenized_prompt = None


    raise NotImplementedError

    tokenized_data["input_ids"], tokenized_data["attention_mask"] = tokenized_data[
        "input_ids"
    ][0], tokenized_data["attention_mask"][0]
    tokenized_prompt["input_ids"], tokenized_prompt["attention_mask"] = (
        tokenized_prompt["input_ids"][0],
        tokenized_prompt["attention_mask"][0],
    )

    # Set the labels to the input IDs
    tokenized_data["labels"] = tokenized_data["input_ids"].clone()

    # TODO: Replace the labels for the prompt tokens with -100.


    raise NotImplementedError

    return tokenized_data

In [ ]:
# <NO_AUTOGRADE> <=== Leave this code cell as is, otherwise the autograder will have trouble processing your submission

In [ ]:
def test_prepare_data_for_distillation():
    # Setup
    student_tokenizer = AutoTokenizer.from_pretrained("gpt2", padding_side="right")
    student_tokenizer.pad_token_id = student_tokenizer.eos_token_id

    # Test 1: Basic functionality
    article = "This is a test article."
    summary = "This is a test summary."
    output = prepare_data_for_distillation(article, summary, student_tokenizer)
    assert "input_ids" in output, "Output should contain input_ids"
    assert "attention_mask" in output, "Output should contain attention_mask"
    assert "labels" in output, "Output should contain labels"
    assert torch.is_tensor(output["input_ids"]), "input_ids should be a tensor"
    assert torch.is_tensor(
        output["attention_mask"]
    ), "attention_mask should be a tensor"
    assert torch.is_tensor(output["labels"]), "labels should be a tensor"

    # Test 2: Don't re-copy TL;DR from summary if it's there
    article = "This is a test article."
    summary = "TL;DR:This is a test summary."
    output = prepare_data_for_distillation(article, summary, student_tokenizer)
    decoded = student_tokenizer.decode(output["input_ids"], skip_special_tokens=True)
    assert decoded == "This is a test article.\nTL;DR:This is a test summary.", \
        "Doesn't handle duplicated TL;DR in a way that matches our implementation" + '\n' + decoded
    summary = "TL;DR: This is a test summary."
    output = prepare_data_for_distillation(article, summary, student_tokenizer)
    decoded = student_tokenizer.decode(output["input_ids"], skip_special_tokens=True)
    assert decoded == "This is a test article.\nTL;DR:This is a test summary.", \
        "Doesn't handle duplicated TL;DR in a way that matches our implementation"

    # Test 3: Correct label masking
    article = "This is a test article."
    summary = "This is a test summary."
    output = prepare_data_for_distillation(article, summary, student_tokenizer)
    input_ids = output["input_ids"]
    labels = output["labels"]

    # Ensure labels for the article part are -100
    article_length = len(student_tokenizer(article + "\nTL;DR:")["input_ids"])
    assert all(
        label == -100 for label in labels[:article_length]
    ), "Labels for article tokens should be -100"

    # Ensure labels for the summary part are not -100
    assert all(
        label != -100 for label in labels[article_length:]
    ), "Labels for summary tokens should not be -100"

    # Test 4: Padding and truncation
    long_article = "This is a very " * 1000  # Create a very long article
    long_summary = "Summary " * 1000
    short_article = "Short."
    short_summary = "Short."
    output = prepare_data_for_distillation(
        long_article, long_summary, student_tokenizer, max_length=1024
    )
    assert (
        output["input_ids"].shape[0] == 1024
    ), "Long inputs should be truncated to max length"

    output = prepare_data_for_distillation(
        short_article, short_summary, student_tokenizer, max_length=1024
    )
    assert (output["attention_mask"][1] != 0).all(), "Short article should no have padding"
    print("All tests passed!")

test_prepare_data_for_distillation()

Let's now prepare the data for distillation.

In [ ]:
kd_dataset = load_from_disk(f"{data_dir}/kd_dataset.hf")

In [ ]:
train_tokenized_data = kd_dataset.map(
    lambda example: prepare_data_for_distillation(
        example["article"],
        example["summary"],
        student_tokenizer,
        max_length=1024,
    ),
    batched=False,
    remove_columns=kd_dataset.column_names,
)
train_tokenized_data

In [ ]:
val_tokenized_data = cnn_dm_cse447_dataset["val"].map(
    lambda example: prepare_data_for_distillation(
        example["article"],
        example["summary"],
        student_tokenizer,
        max_length=1024,
    ),
    batched=False,
    remove_columns=cnn_dm_cse447_dataset["val"].column_names,
)
val_tokenized_data

In [ ]:
# Let's have a look at the tokenized data
tokenized_data = train_tokenized_data[0]
print("Input IDs:")
print(tokenized_data["input_ids"])
print("Labels:")
print(tokenized_data["labels"])
print("***********************\n\n")

print("Input Text:")
pprint(student_tokenizer.decode(tokenized_data["input_ids"], skip_special_tokens=True))

### Fine-tuning the student model

We are now ready to fine-tune the student model. While this might sound like a daunting task, we actually need not write much code for this. Transformers library has a `Trainer` class that takes care of the fine-tuning process. We will use the `TrainingArguments` class to set the training arguments and the `Trainer` class to fine-tune the model. Below is the code to fine-tune the student model.

**If you find yourself getting an OutOfMemory error, try changing `per_device_train_batch_size` and `per_device_eval_batch_size` to 2, and upping `gradient_accumulation_steps` to 8.** While this might move your subsequent results around a bit from the ones we report due to producing a slightly different sequence of random seeded numbers, conceptually this change will result in the same model, just trained a bit more slowly.

You're welcome to change training arguments if you like (except for `report_to`), but regardless of whether you do or not, **leave the output that's generated when the following cell is run as-is; we check it to see whether finetuning successfully completed.**

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

if device.lower().startswith('cuda'):
    try:
        # because otherwise, if we're on a T4 GPU runtime,
        # holding cola_model on the GPU too will lead to OOMemory errors in the code block below
        del cola_model
    except NameError as e:
        pass
    torch.cuda.empty_cache()

set_seed()
student_model = AutoModelForCausalLM.from_pretrained("gpt2")
training_args = TrainingArguments(
    output_dir="checkpoints/",
    logging_steps=1,
    eval_strategy="steps",
    eval_steps=27,
    logging_first_step=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,  # don't decrease this, we check that at least three epochs completed
    weight_decay=0.01,
    fp16=True,
    learning_rate=2e-5,
    warmup_steps=100,
    seed=42,
    report_to="tensorboard",  # LEAVE THIS LINE THE SAME, the autograder needs it
    gradient_checkpointing=True
)

# LEAVE ALL REMAINING CODE IN THIS CODE BLOCK THE SAME, the autograder needs it

# Create a data collator for dynamic padding
data_collator = DataCollatorForSeq2Seq(
    tokenizer=student_tokenizer, model=student_model, padding="longest"
)
trainer = Trainer(
    model=student_model,
    args=training_args,
    train_dataset=train_tokenized_data,
    eval_dataset=val_tokenized_data,
    data_collator=data_collator,
)

trainer.train()
print('Training complete.')

At the end of training you should see a training loss of around 2.44 and validation loss of 3.00 (despite calling `set_seed()`, the exact numbers you get might shift a bit on different training runs, though. So don't be worried if your numbers are a little off from ours).

Let's now generate summaries with the fine-tuned student model and evaluate the performance.

**Leave the code in the following code block, AND the output that's generated once you run it, as-is; the autograder needs both.**

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
set_seed()
# Padding side left for generation
student_tokenizer.padding_side = "left"
pred_summaries_ft_student = summarize_wth_student_model(
    cnn_dm_cse447_dataset["val"]["article"], student_model, student_tokenizer, p=0.9, temperature=1.0, device=device
)


In [ ]:
# Inspect the summaries
print("Article:")
pprint(cnn_dm_cse447_dataset["val"][0]["article"])
print("***********************\n\n")
print("Generated summary:")
pprint(pred_summaries_ft_student[0])
print("***********************\n\n")
print("Gold summary:")
pprint(cnn_dm_cse447_dataset["val"][0]["summary"])

While not perfect, the summaries generated by the fine-tuned student model are much better than the ones generated by the student model before fine-tuning. Also, note that we only used a very small number of examples i.e. 1000 to distill the model, in practice it is common to use a larger dataset. Let's now evaluate the performance of the fine-tuned student model.

**Leave the code in the following code block, AND the output that's generated once you run it, as-is; the autograder needs both.**

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
import evaluate
teacher_rouge = evaluate.load("rouge")
scores = teacher_rouge.compute(
    predictions=pred_summaries_ft_student, references=cnn_dm_cse447_dataset["val"]["summary"], use_stemmer=True
)
print(scores)

In [ ]:
# Leave this code cell as is, otherwise the autograder will have trouble processing your submission
# </NO_AUTOGRADE>

You should see the following scores:
- rouge1: 0.230
- rouge2: 0.057
- rougeL: 0.145
- rougeLsum: 0.183

(Once again, though, you might observe a small deviation from our numbers in this section, even if you did everything correctly, despite us calling `set_seed()` earlier. Don't worry if you do see this small deviation. This kind of slight nondeterminism despite best efforts is a common issue when training on GPUs, unfortunately...)

As you can see, the fine-tuned student model performs better than the student model before fine-tuning. This is impressive because we used very small teacher and student models and only used 1000 examples for distillation.

In practice, you would want to use a much bigger teacher model and a bigger student model to get better performance. The purpose of this exercise is to give you an idea of how distillation works and how you can use it for your own use cases.